In [1]:
import requests
from datetime import datetime
import pandas as pd
import os
from dotenv import load_dotenv

load_dotenv()

cities_list = pd.read_excel('cities_list.xlsx')

API_KEY = os.getenv('API_KEY')

def get_air_pollution_Data(API_KEY, lat, lon):
    url = f'http://api.openweathermap.org/data/2.5/air_pollution/forecast?lat={lat}&lon={lon}&appid={API_KEY}'
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        print(f'Error requisition: {response.status_code}')
        return None

In [2]:
cities_list

,name,lon,lat,country
0,Abidjan,-4.00167,5.35444,Ivory Coast
1,Abu Dhabi,54.39696,24.45118,United Arab Emirates
2,Abuja,7.49508,9.05785,Nigeria
3,Accra,-0.19690,5.55602,Ghana
4,Addis Ababa,38.74689,9.02497,Ethiopia
...,...,...,...,...
366,Yaounde,11.51667,3.86667,Cameroon
367,Yekaterinburg,60.61220,56.85190,Russia
368,Yerevan,44.51361,40.18111,Armenia
369,Yokohama,139.65000,35.43333,Japan


In [3]:
get_air_pollution_Data(API_KEY, -4.00167, 5.35444)

{'coord': {'lon': 5.3544, 'lat': -4.0017},
 'list': [{'main': {'aqi': 2},
   'components': {'co': 83.62,
    'no': 0,
    'no2': 0.02,
    'o3': 44.41,
    'so2': 0.19,
    'pm2_5': 12.54,
    'pm10': 28.97,
    'nh3': 0},
   'dt': 1748574000},
  {'main': {'aqi': 2},
   'components': {'co': 82.73,
    'no': 0,
    'no2': 0.02,
    'o3': 45.28,
    'so2': 0.2,
    'pm2_5': 12.25,
    'pm10': 29.31,
    'nh3': 0},
   'dt': 1748577600},
  {'main': {'aqi': 2},
   'components': {'co': 81.64,
    'no': 0,
    'no2': 0.02,
    'o3': 46.05,
    'so2': 0.2,
    'pm2_5': 11.97,
    'pm10': 29.81,
    'nh3': 0},
   'dt': 1748581200},
  {'main': {'aqi': 2},
   'components': {'co': 80.91,
    'no': 0,
    'no2': 0.02,
    'o3': 46.73,
    'so2': 0.21,
    'pm2_5': 11.76,
    'pm10': 30.43,
    'nh3': 0},
   'dt': 1748584800},
  {'main': {'aqi': 2},
   'components': {'co': 80.21,
    'no': 0,
    'no2': 0.03,
    'o3': 47.27,
    'so2': 0.21,
    'pm2_5': 11.61,
    'pm10': 31.07,
    'nh3': 0},
   

In [26]:
def process_pollution_data(data, city, country):
    if not data or 'list'not in data:
        return None
    
    formatted_data = []
    for item in data['list']:
        row = {
            'dt': item['dt'],
            'aqi': item['main']['aqi'],
            'city': city,
            'country': country
        }
        if isinstance(item.get('components'), dict):
            row.update(item['components'])
        formatted_data.append(row)

    coord = data.get('coord', {})
    if coord:
        row = {
            'lon': coord.get('lon', None),
            'lat': coord.get('lat', None),
            'city': city,  # Keep city and country for consistency
            'country': country
        }
        formatted_data.append(row)

    return formatted_data

In [27]:
all_data = []

In [28]:
for index, row in cities_list.iterrows():
    lon = row.lon
    lat = row.lat
    country = row.country
    city = row.name

    data = get_air_pollution_Data(API_KEY, lat, lon)

    if data:
        city_data = process_pollution_data(data, city, country)
        if city_data:
            all_data.append(city_data)

In [29]:
df = pd.DataFrame(all_data)

In [30]:
columns = ['dt', 'city', 'country', 'co', 'no', 'no2', 'o3', 'so2', 'pm2_5', 'pm10', 'nh3', 'lon', 'lat']

In [33]:
correction_data = [item for sublist in all_data for item in sublist]
df = pd.DataFrame(correction_data)

In [34]:
df

,dt,aqi,city,country,co,no,no2,o3,so2,pm2_5,pm10,nh3,lon,lat
0,1.748574e+09,1.0,0,Ivory Coast,76.85,0.0,0.17,41.60,0.13,3.15,4.39,0.00,NaN,NaN
1,1.748578e+09,1.0,0,Ivory Coast,77.33,0.0,0.16,41.45,0.12,3.14,4.38,0.00,NaN,NaN
2,1.748581e+09,1.0,0,Ivory Coast,77.76,0.0,0.16,41.06,0.12,3.21,4.49,0.00,NaN,NaN
3,1.748585e+09,1.0,0,Ivory Coast,78.20,0.0,0.15,40.78,0.12,3.28,4.59,0.00,NaN,NaN
4,1.748588e+09,1.0,0,Ivory Coast,78.71,0.0,0.19,39.98,0.14,3.35,4.85,0.00,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35982,1.748905e+09,1.0,370,Croatia,98.70,0.0,2.37,54.53,0.21,6.34,7.14,1.06,NaN,NaN
35983,1.748909e+09,1.0,370,Croatia,97.94,0.0,2.01,55.21,0.22,6.19,7.01,1.07,NaN,NaN
35984,1.748912e+09,1.0,370,Croatia,97.41,0.0,1.67,56.83,0.23,6.07,6.94,1.10,NaN,NaN
35985,1.748916e+09,1.0,370,Croatia,97.49,0.0,1.41,59.01,0.25,6.00,6.92,1.16,NaN,NaN
